In [ ]:
from sklearn.linear_model import RidgeCV
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

# from sklearn.pipeline import make_pipeline
# model = make_pipeline(StandardScaler(with_mean=False), _RidgeGCV())

teams_dict, teams_list = get_teams(league="NBA")
box_DIR = "../data/box/"
img_DIR_T = "../data/images/teams/"

In [ ]:
def get_ratings(season=2024,ngames=0):
    cols = [
        "gameId",
        "teamName",
        "teamId",
        "offensiveRating",
        "defensiveRating",
        "netRating",
        "possessions",
    ]
    df = pd.read_parquet(box_DIR + f"NBA_Box_T_Adv_{season}.parquet", columns=cols)
    cols = ["gameId", "team", "tId", "ORtg", "DRtg", "NRtg", "poss"]
    df.columns = cols
    df1 = df.groupby("gameId")
    df1_1 = df1.nth(0)
    df1_2 = df1.nth(1)
    df1_1.columns = ["gameId"] + [s + "1" for s in df1_1.columns if s != "gameId"]
    df1_2.columns = ["gameId"] + [s + "2" for s in df1_2.columns if s != "gameId"]
    df1_3 = pd.merge(df1_1, df1_2, on="gameId")
    df1_4 = df1.nth(1)
    df1_5 = df1.nth(0)
    df1_4.columns = ["gameId"] + [s + "1" for s in df1_4.columns if s != "gameId"]
    df1_5.columns = ["gameId"] + [s + "2" for s in df1_5.columns if s != "gameId"]
    df1_6 = pd.merge(df1_4, df1_5, on="gameId")
    df2 = pd.concat([df1_3, df1_6]).sort_values(by="gameId").reset_index(drop=True)
    data1 = df2.copy()
    df10 = pd.read_parquet(box_DIR + f"NBA_Box_T_Base_{season}.parquet")
    if ngames != 0:
        df10g = df10.groupby("TEAM_NAME")
        df10 = df10g.nth(np.arange(-ngames,0,1)).reset_index(drop=True)
    df10["HOME"] = ~df10["MATCHUP"].str.contains("@")
    df10["tId1"] = df10["TEAM_ID"]
    df10["gameId"] = df10["GAME_ID"]
    df11 = (
        df10[["gameId", "tId1", "HOME"]].sort_values(by="gameId").reset_index(drop=True)
    )
    df11[["gameId", "tId1"]] = df11[["gameId", "tId1"]].astype(int)
    data = pd.merge(data1, df11)
    return data

In [ ]:
def process_results(data, results_adj):
    data["pts1"] = data["ORtg1"] * data["poss1"]
    data["pts2"] = data["DRtg1"] * data["poss1"]
    off_prior = data.groupby(["tId1"])[["poss1", "pts1"]].agg("sum").reset_index()
    def_prior = data.groupby(["tId1"])[["poss1", "pts2"]].agg("sum").reset_index()
    off_prior["OFF"] = off_prior["pts1"] / off_prior["poss1"]
    off_prior = off_prior[["tId1", "OFF"]]
    def_prior["DEF"] = def_prior["pts2"] / def_prior["poss1"]
    def_prior = def_prior[["tId1", "DEF"]]
    results_net = pd.merge(off_prior, def_prior, on=["tId1"])
    results_net["NET"] = results_net["OFF"] - results_net["DEF"]
    results_net.rename(columns={"tId1": "tId"}, inplace=True)
    results_net = results_net.astype(float).round(2)
    results_net["tId"] = results_net["tId"].astype(int)
    ortg_mean = data["pts1"].sum() / data["poss1"].sum()
    drtg_mean = data["pts2"].sum() / data["poss1"].sum()
    results_adj["tId"] = results_adj["tId"].astype(int)
    results_comb = pd.merge(results_net, results_adj, on=["tId"])
    results_comb["aOFF"] = results_comb["aOFF"]
    results_comb["aDEF"] = results_comb["aDEF"]
    results_comb["oSOS"] = results_comb["aOFF"] - results_comb["OFF"]
    results_comb["dSOS"] = results_comb["DEF"] - results_comb["aDEF"]
    results_comb["SOS"] = results_comb["oSOS"] + results_comb["dSOS"]
    results_comb.iloc[:, 1:] = results_comb.iloc[:, 1:].round(1)
    results = results_comb[
        ["Team", "OFF", "oSOS", "aOFF", "DEF", "dSOS", "aDEF", "NET", "SOS", "aNET"]
    ]
    # results = results_comb[["Team","OFF","DEF","NET","aOFF","aDEF","aNET"]]
    results = results.sort_values(by="aNET", ascending=0).reset_index(drop=True)
    return results, ortg_mean, drtg_mean

In [ ]:
def map_teams(row_in, teams, scale):
    t1 = row_in[0]
    t2 = row_in[1]

    rowOut = np.zeros([len(teams) * 2])
    rowOut[teams.index(t1)] = scale
    rowOut[teams.index(t2) + len(teams)] = scale

    return rowOut


def convert_to_matricies(possessions, name, teams, scale=1):
    # extract only the columns we need
    # Convert the columns of player ids into a numpy matrix
    stints_x_base = possessions[["tId1", "tId2"]].to_numpy()
    # Apply our mapping function to the numpy matrix
    stint_X_rows = np.apply_along_axis(map_teams, 1, stints_x_base, teams, scale=scale)
    # Convert the column of target values into a numpy matrix
    stint_Y_rows = possessions[name].to_numpy()

    # return matricies and possessions series
    return stint_X_rows, stint_Y_rows


# Convert lambda value to alpha needed for ridge CV


def lambda_to_alpha(lambda_value, samples):
    return (lambda_value * samples) / 2.0


# Convert RidgeCV alpha back into a lambda value


def alpha_to_lambda(alpha_value, samples):
    return (alpha_value * 2.0) / samples


def calculate_netrtg(train_x, train_y, lambdas, teams_list):
    alphas = [lambda_to_alpha(l, train_x.shape[0]) for l in lambdas]
    # create a 5 fold CV ridgeCV model. Our target data is not centered at 0, so we want to fit to an intercept.
    clf = RidgeCV(alphas=alphas, cv=5, fit_intercept=True)

    # fit our training data
    model = clf.fit(
        train_x,
        train_y,
    )

    # convert our list of players into a mx1 matrix
    team_arr = np.transpose(np.array(teams_list).reshape(1, len(teams_list)))

    # extract our coefficients into the offensive and defensive parts
    coef_offensive_array = model.coef_[0 : len(teams_list)][np.newaxis].T
    coef_defensive_array = model.coef_[len(teams_list) : 2 * len(teams_list)][
        np.newaxis
    ].T
    # concatenate the offensive and defensive values with the playey ids into a mx3 matrix
    team_id_with_coef = np.concatenate(
        [team_arr, coef_offensive_array, coef_defensive_array], axis=1
    )
    # build a dataframe from our matrix
    teams_coef = pd.DataFrame(team_id_with_coef)
    intercept = model.intercept_
    teams_coef.columns = ["tId", "aOFF", "aDEF"]
    teams_coef["aNET"] = teams_coef["aOFF"] - teams_coef["aDEF"]
    teams_coef["aOFF"] = teams_coef["aOFF"] + intercept
    teams_coef["aDEF"] = teams_coef["aDEF"] + intercept
    teams_coef["Team"] = teams_coef["tId"].map(teams_dict)
    results = teams_coef[["tId", "Team", "aOFF", "aDEF", "aNET"]]
    results = results.sort_values(by=["aNET"], ascending=False).reset_index(drop=True)
    return results, model, intercept

In [ ]:
season = 2024
ngames = 0

In [ ]:
datar = get_ratings(season,ngames=0)
train_x, train_y = convert_to_matricies(datar, "ORtg1", teams_list, scale=1 / 2)
n = 1.5*0.1
# n = 1.5/2*0.1
lambdas_net = [0.01 * n, 0.05 * n, 0.1 * n]
# lambdas_net = [10/3]
results_adj, model, intercept = calculate_netrtg(
    train_x, train_y, lambdas_net, teams_list
)
results, ortg_mean, drtg_mean = process_results(datar, results_adj)
results.index = results.index +1
print(intercept)
# results.head(5)
# results.sort_values(by="aOFF",ascending=0)
# results.sort_values(by="aDEF",ascending=1)
# cc

In [ ]:
"https://stats.nba.com/stats/scheduleleaguev2?Season=2024-25&LeagueID=00"
"http://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2018/league/00_full_schedule.json"

In [ ]:
r = requests.get(f"http://data.nba.com/data/10s/v2015/json/mobile_teams/nba/{season}/league/00_full_schedule.json")

In [ ]:
g

In [ ]:
rj = r.json()
gid, tid_v, tid_h, game_date = [], [], [], []
for mo in rj['lscd']:
    for g in mo['mscd']['g']:
        gid.append(g['gid'])
        tid_v.append(g['v']['tid'])
        tid_h.append(g['h']['tid'])
        game_date.append(g['gdte'])

In [ ]:
dfs = pd.DataFrame({'gid':gid,'tId_h':tid_h,'tId_v':tid_v,'game_date':game_date })
dfs["game_date"] = pd.to_datetime(dfs["game_date"])
dfs = dfs[dfs["gid"].str[:3] == "002"]
dfs = dfs.sort_values(["game_date","gid"]).reset_index(drop=True)

In [ ]:
ty = datetime.today()
dfsf = dfs.query(f"game_date >= '{ty.strftime("%b %d, %Y")}'").reset_index(drop=True)

In [ ]:
df11 = dfsf[["gid", "tId_h", "tId_v","game_date"]]
df11["home"] = True
df11 = df11.rename(columns={"tId_h":"tId","tId_v":"tId2"})
df12 = dfsf[["gid", "tId_v", "tId_h","game_date"]]
df12["home"] = False
df12 = df12.rename(columns={"tId_h":"tId2","tId_v":"tId"})
df1 = pd.concat([df11,df12]).sort_values(["game_date","gid","home"]).reset_index(drop=True)

In [ ]:
dfn = results_adj[["tId","aNET"]]
dfn.columns = ["tId2","aNET"]
df2  = pd.merge(df1,dfn)

In [ ]:
df3 = df2.groupby("tId")[["aNET"]].mean()
df3 = df3.reset_index()
df3["Team"] = df3["tId"].map(teams_dict)
df3 = df3.rename(columns={"aNET":"oNET"})
dfn = results_adj[["tId","aNET"]]
df4 = pd.merge(df3,dfn)
df4 = df4[["tId","Team","aNET","oNET"]].sort_values("oNET",ascending=False).reset_index(drop=True)

In [ ]:
df_teams = pd.read_csv("../data/NBA_teams_colors_logos.csv")
df_teams = df_teams.rename(columns={"nameTeam": "Team"})
df_teams = df_teams[["Team","colorsTeam"]]
data = pd.merge(df4, df_teams)
data["image"] = img_DIR_T + data["Team"] + ".png"
teams = data["Team"].to_list()
teams.reverse()
data["Team"] = pd.Categorical(data["Team"],categories=teams,ordered=True)
today = datetime.today().strftime("%B %d, %Y")

In [ ]:
p = (
    ggplot(data)
    + aes(x="Team",y="oNET",fill="colorsTeam",image="image")
    + geom_col(alpha=0.7,show_legend=False)
    + geom_image(aes(y=0),size=0.06)
    + coord_flip()
    + scale_color_identity(aesthetics=["fill"])
    + theme_xkcd(base_size=15)
    + theme(
        figure_size=(10,10),
        text=element_text(family=["Comic Sans MS"])
    )
    + pnba
    + labs(
        title=f"Remaining Strength of Schedule (SoS) as of {today}",
        subtitle="Opponent Net Rating is Adjusted for previous SoS",
        x="",
        y="Remaining Opponent Net Rating"
    )
)
p